<a href="https://colab.research.google.com/github/brendontj/tcc-lol-stats/blob/master/LOL_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import math
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [66]:
df = pd.read_csv('Result_1.csv')
df.head()   

,#,game_id,game_external_ref,match_external_ref,patch_version,blue_team_ref,red_team_ref,game_statem,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,blue_team_dragons,red_team_total_gold,red_team_inhibitors,red_team_towers,red_team_barons,red_team_total_kills,red_team_dragons,participant_id,champion_name,esports_player_ref,summoner_name,role,game_timestamp,level,kills,deaths,assists,total_gold_earned,creep_score,kill_participation,champion_damage_share,wards_placed,wards_destroyed,event_external_ref,game_number,status,team_a_external_ref,team_b_external_ref,team_a_side,team_b_side,match_external_ref.1,team_a_name,team_a_code,team_b_name,team_b_code,team_a_record_wins,team_a_record_losses,team_b_record_wins,team_b_record_losses,team_a_game_wins,team_b_game_wins,best_of,state,league_name,tournament_external_ref,league_external_ref,region
1960,1961,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,0,0,0,0,0,NaN,0,0,0,0,0,NaN,1,Gangplank,1.022063e+17,C9 Fudge,top,2020-08-28 22:03:00.000000,1,0,0,0,0,0,0.000000,0.000000,0,0,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE
1961,1962,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,0,0,0,0,0,NaN,0,0,0,0,0,NaN,2,Evelynn,9.892651e+16,C9 Inori,jungle,2020-08-28 22:03:00.000000,1,0,0,0,0,0,0.000000,0.000000,0,0,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE
1962,1963,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,0,0,0,0,0,NaN,0,0,0,0,0,NaN,3,Lucian,9.910110e+16,C9 Palafox,mid,2020-08-28 22:03:00.000000,1,0,0,0,0,0,0.000000,0.000000,0,0,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE
1963,1964,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,0,0,0,0,0,NaN,0,0,0,0,0,NaN,4,Ashe,1.041510e+17,C9 Tomo,bottom,2020-08-28 22:03:00.000000,1,0,0,0,0,0,0.000000,0.000000,0,0,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE
1964,1965,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,0,0,0,0,0,NaN,0,0,0,0,0,NaN,5,Leona,1.013838e+17,C9 Diamond,support,2020-08-28 22:03:00.000000,1,0,0,0,0,0,0.000000,0.000000,0,0,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,2026,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,58393,1,8,1,18,"{ocean,infernal,mountain,mountain}",44359,0,1,0,3,NaN,6,Jayce,9.932241e+16,TSM Dhokla,top,2020-08-28 22:33:00.000000,14,1,6,0,10309,235,0.333333,0.494185,9,8,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TS

#Utils functions



In [67]:
def get_dragon_names(arr):
  dragons = []
  for i in arr:
    if not pd.isna(i):
      for j in i[1:-1].split(','):
        if j not in dragons:
          dragons.append(j)
  return dragons

def number_of_dragons(arr):
  if arr == 0:
    return 0
  return len(arr[1:-1].split(','))

#Model to predict winner at 05 minutes of game

In [69]:
df_1 = df.copy()
df_1['red_team_dragons'] = df_1['red_team_dragons'].fillna(0)

df_1['blue_team_dragons'] = df_1['blue_team_dragons'].fillna(0)

df_1['len_red_team_dragons'] = df_1['red_team_dragons'].apply(lambda x: number_of_dragons(x))
df_1['Len_blue_team_dragons'] = df_1['blue_team_dragons'].apply(lambda x: number_of_dragons(x))

df_1.head(2000)


,#,game_id,game_external_ref,match_external_ref,patch_version,blue_team_ref,red_team_ref,game_statem,blue_team_total_gold,blue_team_inhibitors,blue_team_towers,blue_team_barons,blue_team_total_kills,blue_team_dragons,red_team_total_gold,red_team_inhibitors,red_team_towers,red_team_barons,red_team_total_kills,red_team_dragons,participant_id,champion_name,esports_player_ref,summoner_name,role,game_timestamp,level,kills,deaths,assists,total_gold_earned,creep_score,kill_participation,champion_damage_share,wards_placed,wards_destroyed,event_external_ref,game_number,status,team_a_external_ref,team_b_external_ref,team_a_side,team_b_side,match_external_ref.1,team_a_name,team_a_code,team_b_name,team_b_code,team_a_record_wins,team_a_record_losses,team_b_record_wins,team_b_record_losses,team_a_game_wins,team_b_game_wins,best_of,state,league_name,tournament_external_ref,league_external_ref,region,len_red_team_dragons,Len_blue_team_dragons
0,1,5c7d0c91-5687-45b2-b31a-9017925e46eb,104151038627427253,104151038627427252,10.16.330.9186,99566406332987990,99566406338282437,in_game,0,0,0,0,0,0,0,0,0,0,0,0,1,Lillia,1.017991e+17,CHF Thien,top,2020-08-22 06:12:00.000000,1,0,0,0,0,0,0.0,0.000000,0,0,104151038627427252,1,completed,99566406332987990,99566406338282437,blue,red,104151038627427252,The Chiefs,CHF,ORDER,ORD,2,1,2,2,1,3,5,completed,OPL,104151038596540368,98767991331560952,OCEANIA,0,0
1,2,5c7d0c91-5687-45b2-b31a-9017925e46eb,104151038627427253,104151038627427252,10.16.330.9186,99566406332987990,99566406338282437,in_game,0,0,0,0,0,0,0,0,0,0,0,0,2,Kayn,1.035252e+17,CHF Croc,jungle,2020-08-22 06:12:00.000000,1,0,0,0,0,0,0.0,0.000000,0,0,104151038627427252,1,completed,99566406332987990,99566406338282437,blue,red,104151038627427252,The Chiefs,CHF,ORDER,ORD,2,1,2,2,1,3,5,completed,OPL,104151038596540368,98767991331560952,OCEANIA,0,0
2,3,5c7d0c91-5687-45b2-b31a-9017925e46eb,104151038627427253,104151038627427252,10.16.330.9186,99566406332987990,99566406338282437,in_game,0,0,0,0,0,0,0,0,0,0,0,0,3,Leblanc,9.956641e+16,CHF Claire,mid,2020-08-22 06:12:00.000000,1,0,0,0,0,0,0.0,0.000000,0,0,104151038627427252,1,completed,99566406332987990,99566406338282437,blue,red,104151038627427252,The Chiefs,CHF,ORDER,ORD,2,1,2,2,1,3,5,completed,OPL,104151038596540368,98767991331560952,OCEANIA,0,0
3,4,5c7d0c91-5687-45b2-b31a-9017925e46eb,104151038627427253,104151038627427252,10.16.330.9186,99566406332987990,99566406338282437,in_game,0,0,0,0,0,0,0,0,0,0,0,0,4,Ashe,1.022063e+17,CHF Katsurii,bottom,2020-08-22 06:12:00.000000,1,0,0,0,0,0,0.0,0.000000,0,0,104151038627427252,1,completed,99566406332987990,99566406338282437,blue,red,104151038627427252,The Chiefs,CHF,ORDER,ORD,2,1,2,2,1,3,5,completed,OPL,104151038596540368,98767991331560952,OCEANIA,0,0
4,5,5c7d0c91-5687-45b2-b31a-9017925e46eb,104151038627427253,104151038627427252,10.16.330.9186,99566406332987990,99566406338282437,in_game,0,0,0,0,0,0,0,0,0,0,0,0,5,Nautilus,1.025460e+17,CHF Dragku,support,2020-08-22 06:12:00.000000,1,0,0,0,0,0,0.0,0.000000,0,0,104151038627427252,1,completed,99566406332987990,99566406338282437,blue,red,104151038627427252,The Chiefs,CHF,ORDER,ORD,2,1,2,2,1,3,5,completed,OPL,104151038596540368,98767991331560952,OCEANIA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,549d364f-431c-45b2-b782-9ea025249f98,104174601840512787,104174601840512784,10.16.330.9186,99566405130451785,99566405127798575,in_game,24845,0,0,0,3,{ocean},22188,0,0,0,0,0,6,Jayce,9.932241e+16,TSM Dhokla,top,2020-08-28 22:18:00.000000,10,0,1,0,5067,125,0.0,0.633026,4,1,104174601840512784,3,completed,99566405130451785,99566405127798575,blue,red,104174601840512784,C9 Academy,C9,TSM Academy,TSM,2,0,1,1,3,1,5,completed,LCS Academy,104174601803063383,99332500638116286,AMÉRICA DO NORTE,0,1
1996,1997,549d364f-431c-45b2-b782-9ea025

{'cloud': 1, 'elder': 2, 'infernal': 3, 'mountain': 4, 'ocean': 5}
